# Batch Optimization Examples

This notebook shows how to run multiple metric/optimizer jobs in a batch using `py_scripts/optimization_workflow.py`.
It also saves audio and plots with a consistent naming scheme matching `save_wav(add_info=True, add_time=True)`.

In [3]:
# Setup and target loading
import numpy as np
import pandas as pd
from IPython.display import display
from pathlib import Path

# Reuse the same configuration as test.ipynb
SR = 44100
DURATION = 2.0
FFT_PAD = 2
FADE_IN_MS = 10.0
FADE_OUT_MS = 10.0

# Load target partials (example: cello)
target_path = 'tsv/voice-single2.tsv'
df = pd.read_csv(target_path, sep="\t")
display(df)
target_freqs = df["Frequency (Hz)"].to_numpy()
target_amps = (df["Amplitude"] / df["Amplitude"].max()).to_numpy()
# Extract a short name for naming (e.g., 'cello_single')
target_name = Path(target_path).stem


,Frequency (Hz),Amplitude
0,431.578947,0.000596
1,444.736842,0.003736
2,452.631579,0.006030
3,460.526316,0.111060
4,478.947368,0.002054
5,905.263158,0.000590
6,921.052632,0.016148
7,926.315789,0.012469
8,939.473684,0.001063
9,1386.842105,0.000786


## Define batch jobs
- `method`: one of `'de'` (Differential Evolution), `'da'` (Dual Annealing), `'bh'` (Basin Hopping).
- `metric`: any supported metric: `'pearson'`, `'mfcc'`, `'itakura_saito'`, `'spectral_convergence'`, `'cosine'`, `'euclidean'`, `'manhattan'`, `'kl'`.
- `kwargs`: optimizer-specific settings (e.g., `maxiter`, `workers`, `stepsize`).

In [4]:
from py_scripts.optimization_workflow import run_batch_jobs, BatchJob

jobs = [
    BatchJob(method='de', metric='pearson', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='pearson', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='pearson', kwargs={'maxiter': 500, 'stepsize': 0.4}),
    
    # Add more combinations as needed, e.g.:
    BatchJob(method='de', metric='mfcc', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='mfcc', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='mfcc', kwargs={'maxiter': 500, 'stepsize': 0.4}),

    BatchJob(method='de', metric='itakura_saito', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='itakura_saito', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='itakura_saito', kwargs={'maxiter': 500, 'stepsize': 0.4}),

    BatchJob(method='de', metric='spectral_convergence', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='spectral_convergence', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='spectral_convergence', kwargs={'maxiter': 500, 'stepsize': 0.4}),

    BatchJob(method='de', metric='cosine', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='cosine', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='cosine', kwargs={'maxiter': 500, 'stepsize': 0.4}),

    BatchJob(method='de', metric='euclidean', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='euclidean', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='euclidean', kwargs={'maxiter': 500, 'stepsize': 0.4}),   

    BatchJob(method='de', metric='manhattan', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='manhattan', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='manhattan', kwargs={'maxiter': 500, 'stepsize': 0.4}),
    
    BatchJob(method='de', metric='kl', kwargs={'maxiter': 500, 'workers': -1}),
    BatchJob(method='da', metric='kl', kwargs={'maxiter': 500}),
    BatchJob(method='bh', metric='kl', kwargs={'maxiter': 500, 'stepsize': 0.4})   

]

rows = run_batch_jobs(
    target_freqs, target_amps, jobs,
    target_name=target_name,
    sr=SR, duration=DURATION, fft_pad=FFT_PAD,
    fade_in_ms=FADE_IN_MS, fade_out_ms=FADE_OUT_MS,
    seed=42,
)

# Summarize batch results (params/history omitted for brevity)
summary = pd.DataFrame([{k: v for k, v in r.items() if k not in ('params','history') } for r in rows])
display(summary.sort_values(['metric', 'best']))


FM 4-osc DE → Pearson:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

[save_wav] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_audio\optimized_output_fm_voice-single2_de_pearson_20250908-161600.wav
TSV file was saved at: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\tsv\final_values_fm_voice-single2_de_pearson_20250908-161600.tsv
[save_time_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_fm_voice-single2_de_pearson_20250908-161600_time.png
[save_spectrum_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_fm_voice-single2_de_pearson_20250908-161600_spectrum.png
[save_error_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_fm_voice-single2_de_pearson_20250908-161600_error.png


FM 4-osc DA → Pearson:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

[save_wav] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_audio\optimized_output_fm_voice-single2_da_pearson_20250908-161640.wav
TSV file was saved at: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\tsv\final_values_fm_voice-single2_da_pearson_20250908-161640.tsv
[save_time_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_fm_voice-single2_da_pearson_20250908-161640_time.png
[save_spectrum_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_fm_voice-single2_da_pearson_20250908-161640_spectrum.png
[save_error_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_fm_voice-single2_da_pearson_20250908-161640_error.png


FM 4-osc BH → Pearson:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

...

FM 4-osc DA → Kl:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

[save_wav] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_audio\optimized_output_fm_voice-single2_da_kl_20250908-164557.wav
TSV file was saved at: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\tsv\final_values_fm_voice-single2_da_kl_20250908-164557.tsv
[save_time_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_fm_voice-single2_da_kl_20250908-164557_time.png
[save_spectrum_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_fm_voice-single2_da_kl_20250908-164557_spectrum.png
[save_error_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_fm_voice-single2_da_kl_20250908-164557_error.png


FM 4-osc BH → Kl:   0%|           0/500 [ETA: ?, Elapsed: 00:00]

[save_wav] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_audio\optimized_output_fm_voice-single2_bh_kl_20250908-164636.wav
TSV file was saved at: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\tsv\final_values_fm_voice-single2_bh_kl_20250908-164636.tsv
[save_time_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_fm_voice-single2_bh_kl_20250908-164636_time.png
[save_spectrum_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_fm_voice-single2_bh_kl_20250908-164636_spectrum.png
[save_error_plot] wrote: c:\Users\egorp\Nextcloud\code\public_repos\FFTimbre\rendered_plots\optimized_output_fm_voice-single2_bh_kl_20250908-164636_error.png


,method,metric,best,wav_path,tsv_path,plots
12,de,cosine,9.160420e-02,rendered_audio\optimized_output_fm_voice-singl...,tsv/final_values_fm_voice-single2_de_cosine_20...,{'time': 'rendered_plots\optimized_output_fm_v...
13,da,cosine,4.867485e-01,rendered_audio\optimized_output_fm_voice-singl...,tsv/final_values_fm_voice-single2_da_cosine_20...,{'time': 'rendered_plots\optimized_output_fm_v...
14,bh,cosine,9.371461e-01,rendered_audio\optimized_output_fm_voice-singl...,tsv/final_values_fm_voice-single2_bh_cosine_20...,{'time': 'rendered_plots\optimized_output_fm_v...
17,bh,euclidean,2.500542e+00,rendered_audio\optimized_output_fm_voice-singl...,tsv/final_values_fm_voice-single2_bh_euclidean...,{'time': 'rendered_plots\optimized_output_fm_v...
15,de,euclidean,5.755499e+04,rendered_audio\optimized_output_fm_voice-singl...,tsv/final_values_fm_voice-single2_de_euclidean...,{'time': 'rendered_plots\optimized_output_fm_v...
16,da,euclidean,5.907275e+04,rendered_audio\optimized_output_fm_voice-singl...,tsv/final_values_fm_voice-single2_da_euclidean...,{'time': 'rendered_plots\optimized_output_fm_v...
6,de,itakura_saito,1.432001e+06,rendered_audio\optimized_output_fm_voice-singl...,tsv/final_values_fm_voice-single2_de_itakura_s...,{'time': 'rendered_plots\optimized_output_fm_v...
7,da,itakura_saito,1.581409e+06,rendered_audio\optimized_output_fm_voice-singl...,tsv/final_values_fm_voice-single2_da_itakura_s...,{'time': 'rendered_plots\optimized_output_fm_v...
8,bh,itakura_saito,1.949517e+06,rendered_audio\optimized_output_fm_voice-singl...,tsv/final_values_fm_voice-single2_bh_itakura_s...,{'time': 'rendered_plots\optimized_output_fm_v...
21,de,kl,1.028224e+00,rendered_audio\optimized_output_fm_voice-singl...,tsv/final_values_fm_voice-single2_de_kl_202509...,{'time': 'rendered_plots\optimized_output_fm_v...


## Outputs and naming schema
Each job saves: 
- Audio (`.wav`) to `rendered_audio/optimized_output_fm_<target>_<method>_<metric>_<YYYYMMDD-HHMMSS>.wav`
- Plots (`.png`) to `rendered_plots/optimized_output_fm_<target>_<method>_<metric>_<YYYYMMDD-HHMMSS>_<type>.png`
  - `<type>` ∈ `time`, `spectrum`, `error`
- Final oscillator values to `tsv/final_values_fm_<target>_<method>_<metric>_<YYYYMMDD-HHMMSS>.tsv`

The naming follows the same `_method_metric_timestamp` pattern used by `save_wav(add_info=True, add_time=True)` for consistency across assets.

## Parallel processing note
- `workers` (parallel execution) is supported by Differential Evolution (`method='de'`).
- Dual Annealing (`'da'`) and Basin Hopping (`'bh'`) do not accept `workers`. Any unsupported kwargs are safely ignored by the batch runner.